In [1]:
import pandas as pd
import numpy as np
from obfuscator import comment_remover, variable_renamer
from csv_stuff import create_labled_table_routine, add_valid_code_columns, remove_given_code
import sys
# sys.path.append('/home/erik/TU/ni/Source-Code-Plagiarism-Detection/Source codes/')
sys.path.append('../../../Source-Code-Plagiarism-Detection/Source codes/')
from remove_given_code_from_student_solution import remove_given_code_from_df


PATH_TEMPLATE = "/home/erik/TU/ni/plagiate_labeltool/data/code_templates/"


def create_plagiate_table(df, semester, ha, task, prog_language, number_labled_pairs, path):
    df_labled = pd.DataFrame(columns=['semester', 'ha', 'task', 'prog_lang',
                                      'surname1', 'lastname1', 'code1', 'code2', 'label'])

    df = add_valid_code_columns(df, semester, ha, [task], prog_language)
    number_of_valid_codes = len(df.loc[df[f'{task} empty'] == 0])
    print("number_of_valid_codes: " + str(number_of_valid_codes))
    plagiate_per_solution = int(np.floor(number_labled_pairs/number_of_valid_codes))
    print("plagiate_per_solution: " + str(plagiate_per_solution))
    for ln, sn, code in df.loc[df[f'{task} empty'] == 0][['Nachname', 'Vorname', task]].values:
        for i in range(plagiate_per_solution):
            plagiate_code = comment_remover(code)
            plagiate_code = variable_renamer(plagiate_code)
            df_labled.loc[len(df_labled)] = [
                semester, ha, task, prog_language, sn, ln, code, plagiate_code, 1]
    df_labled.to_csv(path)
    return df_labled


def create_plagiate_table_routine(semester, ha, task, prog_language):
    csv_path = f'../../data/raw_data/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten.csv'
    csv_path1 = f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_removed_preload.csv'
    # csv_path1 = f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_labled.csv'
    csv_path2 = f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_plagiate.csv'
    df = pd.read_csv(csv_path, delimiter=',')
    number_labled_pairs = len(pd.read_csv(csv_path1, delimiter=','))
    df_labled = create_plagiate_table(
        df, semester, ha, task, prog_language, number_labled_pairs, csv_path2)
    return df_labled


def create_plagiate_table_from_df(df, semester, ha, prog_language):
    save_path = f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_plagiate.csv'
    number_labled_pairs = len(df)
    df = df.drop_duplicates(subset=['surname1'])
    number_of_valid_codes = len(df)
    print("number_of_valid_codes: " + str(number_of_valid_codes))
    print("number_labled_pairs: " + str(number_labled_pairs))
    plagiate_per_solution = int(np.floor(number_labled_pairs/number_of_valid_codes))
    print("plagiate_per_solution: " + str(plagiate_per_solution))
    # drop unnecessary columns
    df = df.drop(columns=['hand_labled', 'Unnamed: 0', 'surname2', 'lastname2'])
    # duplicate each row of df1 plagiate_per_solution times and create new column code2 and use the field code1 as input value for the functions comment_remover and variable_renamer
    df = df.reindex(df.index.repeat(plagiate_per_solution))
    df['code2'] = df['code1'].apply(comment_remover)
    df['code2'] = df['code2'].apply(variable_renamer)
    df['label'] = 1
    df.to_csv(save_path)
    return df


def split_data_from_labled(semester, ha, prog_language):
    df_labled = pd.read_csv(f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_labled.csv', delimiter=',')
    df_labled = remove_given_code_from_df(df_labled)
    df_plagiate = create_plagiate_table_from_df(df_labled, semester, ha, prog_language)
    return df_plagiate
    

def open_csvs(semester, ha, prog_language):
    original_csv_path = f'../../data/raw_data/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten.csv'
    labled_csv_path = f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_removed_preload.csv'
    plagiate_csv_path = f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_plagiate.csv'
    df_original = pd.read_csv(original_csv_path, delimiter=',')
    df_labled = pd.read_csv(labled_csv_path, delimiter=',')
    return df_original, df_labled, plagiate_csv_path


In [2]:
semester='SoSe22'
ha='9'
task='Antwort 9'
prog_language='C'
# path = f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_plagiate.csv'
path = f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_labled_angefangen.csv'

In [3]:
# df_plagiate = split_data_from_labled(semester, ha, prog_language)

In [ ]:
def remove_specific_students(df_labled, names):
    for name in names:
        df_labled = df_labled.drop(df_labled[(df_labled['lastname1'] == name) | (df_labled['lastname2'] == name)].index)
    return df_labled


df_labled = pd.read_csv(path)
try:
    df_labled = df_labled.drop(columns=['Unnamed: 0'])
except KeyError:
    pass
print(df_labled.columns)
names = ['Lehmann', 'Hengst', 'Hilger', 'Krause']
df_labled = remove_specific_students(df_labled, names)

print(df_labled['label'].mean())
df_labled['label'].hist()
df_labled['hand_labled'].hist()
# df_labled.to_csv(path, index=False)
# vorgabe entfernen, auto labeln lassen und dann plagiate erstellen

In [5]:
import re
def striphtml(data): 
    return re.sub('<.*?>', ' ', str(data)) 

def stripunc(data): 
    return re.sub('[^A-Za-z%\._\[\]]+', ' ', str(data), flags=re.MULTILINE|re.DOTALL) 

In [ ]:
# # for prasentation to show example strip
# read_path='/home/erik/TU/ni/plagiarism_detection_tool/data/labled/handmade_plagiate_0_0.c'
# write_path='/home/erik/TU/ni/plagiarism_detection_tool/data/labled/handmade_plagiate_0_3.c'
# with open(read_path, 'r') as file:
#     code = file.read()
#     code = striphtml(stripunc(code))
#     with open(write_path, 'w') as file:
#         file.write(code)

In [10]:
def remove_comments(text, regexs = ["//[\t  \S]*", "/\*[\s\S]*?\*/"]):
    tmp = text
    for regex in regexs :
        all_matches = re.findall(regex,tmp)
        for match in all_matches:
            tmp = tmp.replace(match, "")
    return tmp

## Swap C loops

In [17]:
import re

def swap_loops_type(given_string):
    # go through the string and if 'for' or 'wile' accours store 'for' or 'while' in a list
    loop_types = []
    for match in re.finditer(r'for|while', given_string):
        loop_types.append(match.group())
    print(loop_types)

# call this function without opening curly braces in substring
def idx_closing_brace(given_substring):
    sum_braces = 1
    for i, char in enumerate(given_substring):
        if char == '{':
            sum_braces += 1
        elif char == '}':
            sum_braces -= 1
        if sum_braces == 0:
            return i
    return -1


def for_to_while(given_string):
    pattern = '\n*(\s*)for\s*\((.*?;)(.*?)(;.*?)\)'
    matches = re.findall(pattern, given_string)
    # get all end indexes of all matches of pattern in string
    indexes = [x.end(0) for x in re.finditer(pattern, given_string)]
    # split for loop into 3 parts and pre-spaces
    for index,(spaces, init, cond, incr) in zip(indexes, matches):
        pattern_tmp = 'for\s*\(' + re.escape(''.join([init, cond, incr]))
        cond = cond.strip()
        # remove ';' at the beginning and add it at the end of the of the third part
        incr = incr[1:].strip()
        incr = incr + ';'
        idx_brace = idx_closing_brace(given_string[index+1:])
        # add missing offset from index+1
        idx_brace += index+1
        given_string = given_string[:idx_brace] + re.sub('}', '\t' + incr + '\n' + spaces + '}', given_string[idx_brace:], 1)
        given_string = re.sub(pattern_tmp, init + '\n' + spaces + 'while (' + cond , given_string, 1)
    return given_string

def while_to_for(given_string):
    pattern = 'while\s*\((.*?)\)(?:(?:.|\n)*)\s+(\S+\+\+;)'
    matches = re.findall(pattern, given_string)
    for match in reversed(matches):
        given_string = re.sub(pattern, 'for (;' + match + ';)', given_string, 1)
    return given_string
    # while\s*\((.*?)\)(?:(?:.|\n)*?)\s+(\S+\+\+\s*;|\+\+\S*\s*;|\S*\s*\+=\s*\S*\s*;|\S*\s*=\s*\S*\s*\+\s*\S*\s*;)\s*}

## Testcases

In [1]:
import unittest

# TODO rewrite test functions to test subclasses
# TODO tests all edge cases, because students dont write good code
class Test(unittest.TestCase):
    given_string3 = 'for (int i = 0; i < 10; i++){\n\tprintf("%d", i);\n}'
    given_string2 = 'for (int i = 0; i < 10; i++){\n\tfor (int j = 0; j < 10; j++){\n\t\tprintf("%d", i);\n\t}\n}'
    given_string = 'for (int i = 0; i < 10; i++){\n\tprintf("%d", i);\n}\nint abc = 0;\nwhile (abc < 10){\n\tprintf("%d", abc);\n\tabc++;\n}'

    def test_remove_comments(self):
        self.assertEqual(remove_comments(self.given_string), self.given_string)
        comment_string = '// Funktion die bis 100 zählt\nfor (int i = 0; i < 10; i++){\n\t/* zweite Schleife\n\t   damit wird 10*10 gerechnet */\n\tfor (int j = 0; j < 10; j++){\n\t\tprintf("%d", i);\t// das ist ein printf\n\t}\n}'
        self.assertEqual(remove_comments(comment_string), '\nfor (int i = 0; i < 10; i++){\n\t\n\tfor (int j = 0; j < 10; j++){\n\t\tprintf("%d", i);\t\n\t}\n}')

    # swap_loops_type(given_string)
    # print(index_of_braces(given_string[29:])

    # right amount of spaces in inserted code *check*
    # doesnt work with nested for loops *check*
    def test_for_to_while(self):
        self.assertEqual(for_to_while(self.given_string), 'int i = 0;\nwhile (i < 10){\n\tprintf("%d", i);\n\ti++;\n}\nint abc = 0;\nwhile (abc < 10){\n\tprintf("%d", abc);\n\tabc++;\n}')
        self.assertEqual(for_to_while(self.given_string2), 'int i = 0;\nwhile (i < 10){\n\tint j = 0;\n\twhile (j < 10){\n\t\tprintf("%d", i);\n\t\tj++;\n\t}\n\ti++;\n}')
        # TODO test case if fields are empty
        one_liner = 'for (int i = 0; i < 10; i++)\n\tprintf("%d", i);'
        expected = 'int i = 0;\nwhile (i < 10){\n\tprintf("%d", i)\n\ti++;\n}'
        self.assertEqual(one_liner, expected)
        one_liner = 'int i = 0;\nfor (;i < 10;){\n\tprintf("%d", i);\n\ti++;\n}'
        expected = 'int i = 0;\nwhile (i < 10){\n\tprintf("%d", i)\n\ti++;\n}'
        self.assertEqual(one_liner, expected)

    # TODO would be nice if increment in while is found and inserted in for loop
    # TODO test case if fields are empty
    def test_while_to_for(self):
        self.assertEqual(while_to_for(self.given_string), 'for (int i = 0; i < 10; i++){\n\tprintf("%d", i);\n}\nint abc = 0;\nfor (;abc < 10;){\n\tprintf("%d", abc);\n\tabc++;\n}')
        self.assertEqual(while_to_for(self.given_string2), 'for (int i = 0; i < 10; i++){\n\tfor (int j = 0; j < 10; j++){\n\t\tprintf("%d", i);\n\t}\n}')
        one_liner = 'int i = 0;\nwhile (i < 10){\n\tprintf("%d", i)\n\ti++;\n}'
        expected = 'for (int i = 0; i < 10; i++)\n\tprintf("%d", i);'
        self.assertEqual(one_liner, expected)

    def test_index_of_braces(self):
        self.assertEqual(idx_closing_brace(self.given_string3), 29)
        self.assertEqual(idx_closing_brace(self.given_string2), 60)

unittest.main(argv=[''], verbosity=2, exit=False)


test_for_to_while (__main__.Test) ... ERROR
test_index_of_braces (__main__.Test) ... ERROR
test_remove_comments (__main__.Test) ... ERROR
test_while_to_for (__main__.Test) ... ERROR

ERROR: test_for_to_while (__main__.Test)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_3435/2069494273.py", line 22, in test_for_to_while
    self.assertEqual(for_to_while(self.given_string), 'int i = 0;\nwhile (i < 10){\n\tprintf("%d", i);\n\ti++;\n}\nint abc = 0;\nwhile (abc < 10){\n\tprintf("%d", abc);\n\tabc++;\n}')
NameError: name 'for_to_while' is not defined

ERROR: test_index_of_braces (__main__.Test)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_3435/2069494273.py", line 41, in test_index_of_braces
    self.assertEqual(idx_closing_brace(self.given_string3), 29)
NameError: name 'idx_closing_brace' is not defined

ERROR: test_remove_com

## regex101
```
\n*(\s*)for\s*\((.*?;)(.*?)(;.*?)\)
```

```
for (; i < 10;i++){
    for (int j = 0; j < 10; j++){
        printf("%d", i);
    }
}
int abc = 0;
while (abc < 10){
    printf("%d", abc);
    abc++;
}

while (abc < 10){
    printf("%d", abc);
    ++abc;
}

while (abc < 10){
    printf("%d", abc);
    abc += 1;
}

while (abc < 10){
    printf("%d", abc);
    abc = abc + 1 ;
}
```

## Swap positions of functions

## Swap positions of functions

## Swap positions variables in comparisons

## Use de morgan's law on logic equations